## Introducción a CNNs

### Referencias
- LeCun, Y., Bengio, Y. and Hinton, G., 2015. [Deep Learning](https://www.nature.com/articles/nature14539). nature, 521(7553), pp.436-444.
- [Ignite documentation](https://pytorch-ignite.ai/)
- [PyTorch tutorials](https://pytorch.org/tutorials/) 
- [torchmetrics examples](https://torchmetrics.readthedocs.io/en/stable/pages/quickstart.html)
- [visdom](https://github.com/fossasia/visdom)


![pytorch cheatsheet](./figs/pytorch-cheat.jpeg "pytorch cheatsheet")

Figure from [pytorch forum](https://discuss.pytorch.org/t/pytorch-cheat-sheet/72016)

### Que pasos debemos seguir para entrenar una CNN?
- Cargar datos, transformarlos en formato necesario para pytorch. $t_{train}$ va a tener transformaciones diferentes a $t_{test}$, por que? 
- A los datos de entrenamiento debemos dividirlos una vez mas para usar elementos de validación durante el entrenamiento.
- Definir arquitectura de nuestra red (Ver models.py)
- Definir métodos funciones de error, optimizador y métricas de evaluación.

In [1]:
from torchvision import datasets
from torchvision import transforms as tfs
from torch.utils import data
import PIL

transforms_train = [
    tfs.RandomHorizontalFlip(p=0.7),
    tfs.RandomAffine(0, scale=(0.7, 1.0)),
    tfs.Resize((64, 64)),
    tfs.Grayscale(1),
    tfs.Lambda(lambda x: PIL.ImageOps.invert(x)),
    tfs.ToTensor(),
]

transforms_test = [
    tfs.RandomHorizontalFlip(p=0.7),
    tfs.RandomAffine(0, scale=(0.7, 1.0)),
    tfs.Resize((64, 64)),
    tfs.Grayscale(1),
    tfs.Lambda(lambda x: PIL.ImageOps.invert(x)),
    tfs.ToTensor(),
]

train_data = datasets.ImageFolder(
    "../data/train/", transform=tfs.Compose(transforms_train)
)

test_data = datasets.ImageFolder(
    "../data/test/", transform=tfs.Compose(transforms_test)
)

In [2]:
len(train_data.samples), len(test_data.samples), round(963 * 0.1)

(963, 319, 96)

In [3]:
splits = data.random_split(train_data, [867, 96])

train_loader = data.DataLoader(splits[0], batch_size=64, shuffle=True)
val_loader = data.DataLoader(splits[1], batch_size=64, shuffle=True)
test_loader = data.DataLoader(test_data, batch_size=64, shuffle=True)

### Ahora tenemos que definir modelo, función de error, optimizador, etc.
- Vamos a ver como usar ignite para entrenamiento supervisado, no se olviden de ver el dashboard the visdom en [http://localhost:8097/](http://localhost:8097/)
- y un simple for loop con torch.


In [4]:
import torch.optim as optim
from torch import nn
from models import _C
import visdom
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
import torch.nn.functional as F
import torch
from aux import create_plot_window
import numpy as np

In [9]:
from torchsummary import summary

C = _C(input_h_w=64)
summary(C, (1, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,088
              ReLU-2           [-1, 64, 32, 32]               0
            Conv2d-3          [-1, 128, 16, 16]         131,200
       BatchNorm2d-4          [-1, 128, 16, 16]             256
              ReLU-5          [-1, 128, 16, 16]               0
            Linear-6                 [-1, 1024]      33,555,456
       BatchNorm1d-7                 [-1, 1024]           2,048
              ReLU-8                 [-1, 1024]               0
            Linear-9                   [-1, 11]          11,275
          Softmax-10                   [-1, 11]               0
Total params: 33,701,323
Trainable params: 33,701,323
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.02
Forward/backward pass size (MB): 1.77
Params size (MB): 128.56
Est

In [8]:
vis = visdom.Visdom()

Setting up a new session...


In [23]:
criterion = nn.CrossEntropyLoss()
if torch.cuda.is_available():
    C = C.cuda()
    criterion = criterion.cuda()

C_optimizer = optim.Adam(C.parameters(), lr=0.0002, betas=(0.5, 0.999))
log_interval = 10

trainer = create_supervised_trainer(C, C_optimizer, F.cross_entropy)
evaluator = create_supervised_evaluator(
    C, metrics={"accuracy": Accuracy(), "ce_ll": Loss(F.cross_entropy)}, device="cpu"
)

train_avg_loss_window = create_plot_window(
    vis, "#Iterations", "Loss", "Training Average Loss"
)
train_avg_accuracy_window = create_plot_window(
    vis, "#Iterations", "Accuracy", "Training Average Accuracy"
)
val_avg_loss_window = create_plot_window(
    vis, "#Epochs", "Loss", "Validation Average Loss"
)
val_avg_accuracy_window = create_plot_window(
    vis, "#Epochs", "Accuracy", "Validation Average Accuracy"
)


@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(engine):
    evaluator.run(train_loader)
    metrics = evaluator.state.metrics
    avg_accuracy = metrics["accuracy"]
    avg_nll = metrics["ce_ll"]
    print(
        "Training Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}".format(
            engine.state.epoch, avg_accuracy, avg_nll
        )
    )
    vis.line(
        X=np.array([engine.state.epoch]),
        Y=np.array([avg_accuracy]),
        win=train_avg_accuracy_window,
        update="append",
    )
    vis.line(
        X=np.array([engine.state.epoch]),
        Y=np.array([avg_nll]),
        win=train_avg_loss_window,
        update="append",
    )


@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(engine):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    avg_accuracy = metrics["accuracy"]
    avg_nll = metrics["ce_ll"]
    print(
        "Validation Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}".format(
            engine.state.epoch, avg_accuracy, avg_nll
        )
    )
    vis.line(
        X=np.array([engine.state.epoch]),
        Y=np.array([avg_accuracy]),
        win=val_avg_accuracy_window,
        update="append",
    )
    vis.line(
        X=np.array([engine.state.epoch]),
        Y=np.array([avg_nll]),
        win=val_avg_loss_window,
        update="append",
    )


trainer.run(train_loader, max_epochs=200)

Setting up a new session...


Training Results - Epoch: 1  Avg accuracy: 0.60 Avg loss: 2.27
Validation Results - Epoch: 1  Avg accuracy: 0.66 Avg loss: 2.26
Training Results - Epoch: 2  Avg accuracy: 0.67 Avg loss: 2.05
Validation Results - Epoch: 2  Avg accuracy: 0.70 Avg loss: 2.04
Training Results - Epoch: 3  Avg accuracy: 0.71 Avg loss: 1.87
Validation Results - Epoch: 3  Avg accuracy: 0.74 Avg loss: 1.84
Training Results - Epoch: 4  Avg accuracy: 0.72 Avg loss: 1.84
Validation Results - Epoch: 4  Avg accuracy: 0.76 Avg loss: 1.83
Training Results - Epoch: 5  Avg accuracy: 0.77 Avg loss: 1.78
Validation Results - Epoch: 5  Avg accuracy: 0.81 Avg loss: 1.74
Training Results - Epoch: 6  Avg accuracy: 0.77 Avg loss: 1.78
Validation Results - Epoch: 6  Avg accuracy: 0.79 Avg loss: 1.76
Training Results - Epoch: 7  Avg accuracy: 0.79 Avg loss: 1.76
Validation Results - Epoch: 7  Avg accuracy: 0.83 Avg loss: 1.72
Training Results - Epoch: 8  Avg accuracy: 0.81 Avg loss: 1.75
Validation Results - Epoch: 8  Avg accura

Validation Results - Epoch: 64  Avg accuracy: 0.90 Avg loss: 1.66
Training Results - Epoch: 65  Avg accuracy: 0.95 Avg loss: 1.60
Validation Results - Epoch: 65  Avg accuracy: 0.91 Avg loss: 1.64
Training Results - Epoch: 66  Avg accuracy: 0.93 Avg loss: 1.62
Validation Results - Epoch: 66  Avg accuracy: 0.92 Avg loss: 1.65
Training Results - Epoch: 67  Avg accuracy: 0.94 Avg loss: 1.61
Validation Results - Epoch: 67  Avg accuracy: 0.91 Avg loss: 1.64
Training Results - Epoch: 68  Avg accuracy: 0.94 Avg loss: 1.61
Validation Results - Epoch: 68  Avg accuracy: 0.92 Avg loss: 1.65
Training Results - Epoch: 69  Avg accuracy: 0.96 Avg loss: 1.60
Validation Results - Epoch: 69  Avg accuracy: 0.91 Avg loss: 1.65
Training Results - Epoch: 70  Avg accuracy: 0.94 Avg loss: 1.61
Validation Results - Epoch: 70  Avg accuracy: 0.92 Avg loss: 1.64
Training Results - Epoch: 71  Avg accuracy: 0.95 Avg loss: 1.60
Validation Results - Epoch: 71  Avg accuracy: 0.89 Avg loss: 1.66
Training Results - Epoch

Validation Results - Epoch: 127  Avg accuracy: 0.93 Avg loss: 1.62
Training Results - Epoch: 128  Avg accuracy: 0.96 Avg loss: 1.59
Validation Results - Epoch: 128  Avg accuracy: 0.91 Avg loss: 1.64
Training Results - Epoch: 129  Avg accuracy: 0.96 Avg loss: 1.58
Validation Results - Epoch: 129  Avg accuracy: 0.93 Avg loss: 1.62
Training Results - Epoch: 130  Avg accuracy: 0.97 Avg loss: 1.57
Validation Results - Epoch: 130  Avg accuracy: 0.92 Avg loss: 1.64
Training Results - Epoch: 131  Avg accuracy: 0.96 Avg loss: 1.58
Validation Results - Epoch: 131  Avg accuracy: 0.92 Avg loss: 1.64
Training Results - Epoch: 132  Avg accuracy: 0.97 Avg loss: 1.58
Validation Results - Epoch: 132  Avg accuracy: 0.90 Avg loss: 1.65
Training Results - Epoch: 133  Avg accuracy: 0.97 Avg loss: 1.57
Validation Results - Epoch: 133  Avg accuracy: 0.93 Avg loss: 1.64
Training Results - Epoch: 134  Avg accuracy: 0.97 Avg loss: 1.57
Validation Results - Epoch: 134  Avg accuracy: 0.91 Avg loss: 1.65
Training 

Training Results - Epoch: 190  Avg accuracy: 0.98 Avg loss: 1.57
Validation Results - Epoch: 190  Avg accuracy: 0.88 Avg loss: 1.67
Training Results - Epoch: 191  Avg accuracy: 0.98 Avg loss: 1.57
Validation Results - Epoch: 191  Avg accuracy: 0.91 Avg loss: 1.65
Training Results - Epoch: 192  Avg accuracy: 0.98 Avg loss: 1.56
Validation Results - Epoch: 192  Avg accuracy: 0.90 Avg loss: 1.65
Training Results - Epoch: 193  Avg accuracy: 0.97 Avg loss: 1.57
Validation Results - Epoch: 193  Avg accuracy: 0.92 Avg loss: 1.62
Training Results - Epoch: 194  Avg accuracy: 0.98 Avg loss: 1.57
Validation Results - Epoch: 194  Avg accuracy: 0.92 Avg loss: 1.64
Training Results - Epoch: 195  Avg accuracy: 0.98 Avg loss: 1.56
Validation Results - Epoch: 195  Avg accuracy: 0.93 Avg loss: 1.62
Training Results - Epoch: 196  Avg accuracy: 0.97 Avg loss: 1.57
Validation Results - Epoch: 196  Avg accuracy: 0.94 Avg loss: 1.61
Training Results - Epoch: 197  Avg accuracy: 0.98 Avg loss: 1.56
Validation 

State:
	iteration: 2800
	epoch: 200
	epoch_length: 14
	max_epochs: 200
	output: 1.550063967704773
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

### Loop de entrenamiento con Torch

In [15]:
from torch.autograd import Variable

epochs = 100

for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, label) in enumerate(train_loader, 0):
        inputs = Variable(inputs)

        # Forward Pass
        scores = C(inputs)
        loss = criterion(scores, label)

        # Backward Pass
        C_optimizer.zero_grad()
        loss.backward()
        C_optimizer.step()

        running_loss += loss.data
        if i % 64 == 63:  # print every 64 mini-batches
            print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1, running_loss / 64))
            running_loss = 0.0


KeyboardInterrupt



In [26]:
torch.save(C.state_dict(), "./weights/classifier.pkl")

### Métricas sobre datos de Evaluación
- Cargar red entrenada
- Matriz de confusion
- Métricas de clasificación 


In [6]:
cla = _C(input_h_w=64)
cla.load_state_dict(torch.load("./weights/classifier.pkl", map_location="cpu"))

<All keys matched successfully>

In [11]:
import torchnet
from torch.autograd import Variable

# create visualization in visdom side
confusion_window = create_plot_window(
    vis, "True Labels", "Predicted", "Confusion Matrix"
)
confusion_matrix = torchnet.meter.ConfusionMeter(11, normalized=True)

for ii, data_ in enumerate(test_loader):
    input_, label = data_
    val_input = Variable(input_)  # .cuda()
    val_label = Variable(label.type(torch.LongTensor))  # .cuda()
    score = cla(val_input)
    confusion_matrix.add(score.data.squeeze(), label.type(torch.LongTensor))

np.set_printoptions(precision=3)
# print(confusion_matrix.value())
print(confusion_matrix.conf)

# using same visdom backend
vis.heatmap(confusion_matrix.value(), win=confusion_window)

[[ 8  1  0  0  0  0  0  0  0  0  0]
 [ 1 86  0  0  0  0  0  0  0  0  0]
 [ 0  0 23  0  2  0  1  0  0  0  0]
 [ 0  0  0  8  1  0  0  0  0  5  0]
 [ 0  1  0  2 16  0  0  0  1  0  0]
 [ 0  0  0  1  0  2  0  0  0  0  0]
 [ 0  0  2  0  0  0 14  0  0  0  0]
 [ 2  0  0  0  0  0  1  9  0  5  0]
 [ 0  1  1  0  1  1  0  0 13  3  0]
 [ 1  0  0  2  2  0  0  1  0 95  0]
 [ 0  0  0  0  0  0  0  0  0  0  6]]


'window_3bdd0317e312d8'

In [13]:
from aux import iterations_test
from sklearn import metrics

y_real, y_pred = iterations_test(cla, test_loader)
print(metrics.classification_report(np.array(y_pred), np.array(y_real)))

/Users/celiacintas/Code/.venvs/generative/lib/python3.9/site-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


              precision    recall  f1-score   support

           0       1.00      0.82      0.90        11
           1       0.98      0.97      0.97        88
           2       0.88      0.85      0.87        27
           3       0.43      0.46      0.44        13
           4       0.75      0.71      0.73        21
           5       0.67      1.00      0.80         2
           6       0.81      0.87      0.84        15
           7       0.53      0.69      0.60        13
           8       0.70      0.78      0.74        18
           9       0.88      0.84      0.86       106
          10       0.83      1.00      0.91         5

    accuracy                           0.85       319
   macro avg       0.77      0.82      0.79       319
weighted avg       0.86      0.85      0.85       319



### Validación cruzada con ignite

In [ ]:
# todo https://pytorch-ignite.ai/how-to-guides/07-cross-validation/

###  Ejercicios

- Visualizar las funciones de perdida para distintos learning rates. Qué observan en los gráficos?
- Visualizar las funciones de perdida para distintos tamanos de lotes. 
- Visualizar un grupo de elementos del grupo de evaluacion con la clasificación y su puntaje respectivo.
- Comparar metricas de red entrenada en 10, 50 y 100 epochs. Qué diferencias observan?
- Re-entrenar un modelo preentrando `from torchvision import  models` para clasificacion, comparar sus resultados.